In [1]:
import csv
import numpy as np
import pandas as pd
import itertools
import folium
import time
import openrouteservice as ors
import sys
import urllib.parse
import pulp

filename = 'test1.csv'
with open(filename, encoding='utf8', newline='') as f:
    csvreader = csv.reader(f)
    place_list = [row for row in csvreader]
place_list

[['地名', 'latitude', 'longitude'],
 ['長津田駅', '35.5319589', '139.49173'],
 ['河口湖ビジネス&リゾート+サワホテル', '35.4946247', '138.7589732'],
 ['Bakery&Table+箱根（ベーカリー＆テーブル）', '35.2001708', '139.0321096'],
 ['うなぎ+桜家', '35.1191933', '138.9138269'],
 ['箱根神社', '35.2040652', '139.0266838'],
 ['大涌谷', '35.2414681', '139.0227329'],
 ['早雲山駅', '35.2459732', '139.0357894'],
 ['箱根ガラスの森美術館', '35.2609093', '139.0183711'],
 ['三嶋大社', '35.1195374', '138.9178982'],
 ['三島スカイウォーク', '35.1547315', '138.9820768']]

In [ ]:
datalist = []

ors_key = "5b3ce3597851110001cf624804ffaeec7cd246038d01eb4d3a32f633"
ors_boot = ors.Client(ors_key)

for i in range(1,len(place_list)-1):
    pos_1 = float(place_list[i][1]),float(place_list[i][2])
    for j in range(2,len(place_list)):
        pos_2 = float(place_list[j][1]),float(place_list[j][2])
        p1r = tuple(reversed(pos_1))
        p2r = tuple(reversed(pos_2))
        
        try:
            route_rslt = ors_boot.directions((p1r,p2r),profile = "driving-car")
            datalist.append([i,j,float(route_rslt["routes"][0]["summary"]["duration"])])
            print(i,j,float(route_rslt["routes"][0]["summary"]["duration"]))
        except:
            pass
tfr_time = pd.DataFrame(datalist, columns=["出発地点","行先","移動時間[s]"])


In [ ]:
tfr_time.to_csv("time1.csv",index=0)

In [54]:
tfr_time = pd.read_csv("time1.csv")

customer_count = len(place_list)-1
loc_loc_time = tfr_time.to_numpy().tolist()
num_places_time = len(tfr_time)

cost = [[86401 for i in range(customer_count)] for j in range(customer_count)]
for i in range(num_places_time):
    cost[int(loc_loc_time[i][0])-1][int(loc_loc_time[i][1])-1] = float(loc_loc_time[i][2])
    cost[int(loc_loc_time[i][1])-1][int(loc_loc_time[i][0])-1] = float(loc_loc_time[i][2])
cost = np.array(cost)
cost

array([[8.6401e+04, 4.9699e+03, 4.5809e+03, 4.3321e+03, 4.6179e+03,
        8.6401e+04, 4.2701e+03, 4.0214e+03, 4.3665e+03, 4.7723e+03],
       [4.9699e+03, 8.6401e+04, 3.4610e+03, 2.8882e+03, 3.5799e+03,
        8.6401e+04, 2.8957e+03, 2.6471e+03, 2.9226e+03, 3.3274e+03],
       [4.5809e+03, 3.4610e+03, 8.6401e+04, 1.4439e+03, 3.1190e+02,
        8.6401e+04, 6.2470e+02, 9.0710e+02, 1.4305e+03, 1.1055e+03],
       [4.3321e+03, 2.8882e+03, 1.4439e+03, 8.6401e+04, 1.6517e+03,
        8.6401e+04, 1.9816e+03, 2.0352e+03, 3.4400e+01, 6.5690e+02],
       [4.6179e+03, 3.5799e+03, 3.1190e+02, 1.6517e+03, 8.6401e+04,
        8.6401e+04, 6.6170e+02, 9.4410e+02, 1.5295e+03, 1.3135e+03],
       [8.6401e+04, 8.6401e+04, 8.6401e+04, 8.6401e+04, 8.6401e+04,
        8.6401e+04, 8.6401e+04, 8.6401e+04, 8.6401e+04, 8.6401e+04],
       [4.2701e+03, 2.8957e+03, 6.2470e+02, 1.9816e+03, 6.6170e+02,
        8.6401e+04, 8.6401e+04, 3.4190e+02, 1.9683e+03, 1.6433e+03],
       [4.0214e+03, 2.6471e+03, 9.0710e+0

In [55]:
lim_time = 3*60*60
count_place = customer_count
place_remove = list(range(count_place))



# 問題の宣言
problem = pulp.LpProblem("CVRP", pulp.LpMaximize)

# 決定変数
x = [[pulp.LpVariable("x%s_%s"%(i,j), cat="Binary") if i != j else None for j in range(count_place)] for i in range(count_place)]

# 目的関数
problem += pulp.lpSum(x[i][j] if i != j else 0
                        for j in range(count_place) for i in range (count_place))


# 制約
# (2)式，各顧客の場所に訪れるのは1台の車両で1度である
for j in range(1, count_place):
    problem += pulp.lpSum(x[i][j] if i != j else 0 for i in range(count_place)) <= 1 

#(3)式, depotから出発して，depotに戻ってくる

# デポを出発した運搬車が必ず 1つの顧客から訪問を開始することを保証する制約条件
problem += pulp.lpSum(x[0][j] for j in range(1,count_place)) == 1
# 必ず 1 つの顧客から運搬車がデポへ到着すること保証する制約条件
problem += pulp.lpSum(x[i][1] for i in range(count_place)) == 1         #############ここを変える
problem += pulp.lpSum(x[1][j] for j in range(count_place)) == 0         #############ここを変える

problem += pulp.lpSum(x[i][0] for i in range(1,count_place)) == 0

#(4)式, ある顧客の所に来る車両数と出る車両数が同じ

for j in range(2,count_place):         #############ここを変える
    problem += pulp.lpSum(x[i][j] if i != j else 0 for i in range(count_place)) -  pulp.lpSum(x[j][i] for i in range(count_place)) == 0

#(5)式, 各車両において最大容量を超えない

problem += pulp.lpSum((3600 + cost[i][j]) * x[i][j]if i != j else 0 for i in range(count_place) for j in range (1,count_place)) <= lim_time +3600


#(6)式, 部分巡回路除去制約
subtours = []
for i in range(2,count_place):
        subtours += itertools.combinations(range(1,count_place), i)

for s in subtours:
    problem += pulp.lpSum(x[i][j] if i !=j else 0 for i, j in itertools.permutations(s,2)) <= len(s) - 1


if problem.solve() == 1:
    print('目的関数値:', pulp.value(problem.objective))

目的関数値: 2.0


In [56]:
basyo_num_list = []
basyo_list = [0]
jikan = 0


for i in range(customer_count):
    for j in range(customer_count):
        if i != j and pulp.value(x[i][j]) == 1:
            print(i,j)
            jikan += 3600 + cost[i][j]
            basyo_num_list.append([i,j])


print(basyo_num_list)
print("時間",lim_time/3600,"\n最適化",(jikan-3600)/3600)

for i in range(len(basyo_num_list)):
    for j in range(len(basyo_num_list)):
        if basyo_num_list[j][0] == basyo_list[-1]:
            basyo_list.append(basyo_num_list[j][1])
basyo_list

0 7
7 1
[[0, 7], [7, 1]]
時間 3.0 
最適化 2.852361111111111


[0, 7, 1]